In [ ]:
%matplotlib inline

from datetime import timedelta, date
import numpy as np
import xarray as xr

In [ ]:
# Open dataset without dask
ds = xr.open_dataset("/scratch/rd/nen/perClaudia/era5/fwi.nc")

In [ ]:
# Extract data array (1 variable)
fwi = ds['fwi']
fwi

In [ ]:
# Generate daily climatological maps corresponding to the daily 90th percentile 

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# Initialise
danger_threshold_maps = []
# Use a random leap year (day and month are used, year is ignored)
start_date = date(2020, 1, 1)
end_date = date(2021, 1, 1)
for single_date in daterange(start_date, end_date):
    print(single_date)
    # Find indices corresponding to a given date
    idx = np.concatenate(np.where((fwi.time.dt.day == single_date.day) & (fwi.time.dt.month == single_date.month)))
    indices_list = []
    for i in idx:
        indices_list.append(list(range(i - 4, i + 4 + 1)))
    # Concatenate all the indices in a 1-dimensional array
    indices = np.concatenate(indices_list)
    # Remove negative indices
    indices = indices[indices >= 0]
    # Remove indices above the time length
    indices = indices[indices <= (fwi.shape[0] - 1)] # last element is non inclusive
    # Calculate the maps of daily 90th percentile as maps of danger thresholds
    fwi_daily_clima = fwi[indices,:,:]
    daily_danger_threshold_map = fwi_daily_clima.quantile(q = 0.90, dim = 'time', keep_attrs = True)
    danger_threshold_maps.append(daily_danger_threshold_map)

# Concatenate
combined = xr.concat(danger_threshold_maps, dim = 'time')
combined

In [ ]:
combined.to_netcdf(path = '/scratch/rd/nen/perClaudia/era5/fwi_era5_1980_2018_90th_daily_clima.nc')